In [407]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from decouple import config
import decouple
config = decouple.AutoConfig('g:\PythonPractice\python practice.py\guvi\projects\Local Food Wastage Management System')



<>:7: SyntaxWarning: invalid escape sequence '\P'
<>:7: SyntaxWarning: invalid escape sequence '\P'
C:\Users\OWNER\AppData\Local\Temp\ipykernel_9112\3924297067.py:7: SyntaxWarning: invalid escape sequence '\P'
  config = decouple.AutoConfig('g:\PythonPractice\python practice.py\guvi\projects\Local Food Wastage Management System')


Data preprocessing 

In [408]:
data_claims = pd.read_csv('data/claims_data.csv')
data_food_listings = pd.read_csv('data/food_listings_data.csv')
data_providers = pd.read_csv('data/providers_data.csv')
data_receivers = pd.read_csv('data/receivers_data.csv')

In [409]:
data_receivers.head()
data_receivers.info()
data_receivers.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Receiver_ID  1000 non-null   int64 
 1   Name         1000 non-null   object
 2   Type         1000 non-null   object
 3   City         1000 non-null   object
 4   Contact      1000 non-null   object
dtypes: int64(1), object(4)
memory usage: 39.2+ KB


,Receiver_ID
count,1000.000000
mean,500.500000
std,288.819436
min,1.000000
25%,250.750000
50%,500.500000
75%,750.250000
max,1000.000000


In [410]:
data_providers.head()
data_providers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Provider_ID  1000 non-null   int64 
 1   Name         1000 non-null   object
 2   Type         1000 non-null   object
 3   Address      1000 non-null   object
 4   City         1000 non-null   object
 5   Contact      1000 non-null   object
dtypes: int64(1), object(5)
memory usage: 47.0+ KB


In [411]:
data_food_listings.head()
# data_food_listings['Food_Type'].value_counts()
data_food_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Food_ID        1000 non-null   int64 
 1   Food_Name      1000 non-null   object
 2   Quantity       1000 non-null   int64 
 3   Expiry_Date    1000 non-null   object
 4   Provider_ID    1000 non-null   int64 
 5   Provider_Type  1000 non-null   object
 6   Location       1000 non-null   object
 7   Food_Type      1000 non-null   object
 8   Meal_Type      1000 non-null   object
dtypes: int64(3), object(6)
memory usage: 70.4+ KB


In [412]:
data_claims.head()
data_claims.info()
data_claims.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Claim_ID     1000 non-null   int64 
 1   Food_ID      1000 non-null   int64 
 2   Receiver_ID  1000 non-null   int64 
 3   Status       1000 non-null   object
 4   Timestamp    1000 non-null   object
dtypes: int64(3), object(2)
memory usage: 39.2+ KB


,Claim_ID,Food_ID,Receiver_ID
count,1000.000000,1000.000000,1000.000000
mean,500.500000,502.163000,492.437000
std,288.819436,292.462255,284.487869
min,1.000000,1.000000,1.000000
25%,250.750000,233.750000,256.000000
50%,500.500000,504.500000,480.500000
75%,750.250000,762.000000,742.250000
max,1000.000000,999.000000,1000.000000


Null and duplicate remove


In [413]:
data_claims.isnull().sum()    #None
data_claims.duplicated().sum()    #None


np.int64(0)

In [414]:
data_food_listings.isnull().sum()   #None
data_food_listings.duplicated().sum()   #None

np.int64(0)

In [415]:
data_providers.isnull().sum()   #None
data_providers.duplicated().sum()   #None

np.int64(0)

In [416]:
data_receivers.isnull().sum()   #None
data_receivers.duplicated().sum()   #None

np.int64(0)

Mysql connection

In [434]:
import MySQLdb

try:
    db = MySQLdb.connect(
        host=config('db_host'),
        user=config("db_user"),
        passwd=config('db_password'),
        port=int(config('db_port')),
        # database='food_wastage_management_system'
    )
    cursor = db.cursor()
    print("Connected to the database successfully!")
except MySQLdb.Error as e:
    print(f"Error connecting to database: {e}")



Connected to the database successfully!


create databases

In [418]:
cursor.execute("CREATE DATABASE IF NOT EXISTS food_wastage_management_system")
db.commit()

In [438]:
query_use_db = "USE food_wastage_management_system"
cursor.execute(query_use_db)

# Creating  Table

cursor.execute("""
    CREATE TABLE IF NOT EXISTS providers (
        Provider_ID INT PRIMARY KEY,
        Name VARCHAR(255),
        Type VARCHAR(100),
        Address TEXT,
        City VARCHAR(100),
        Contact VARCHAR(50)
    )
""")
cursor.execute("""
    CREATE TABLE IF NOT EXISTS receivers (
        Receiver_ID INT PRIMARY KEY,
        Name VARCHAR(255),
        Type VARCHAR(100),
        City VARCHAR(100),
        Contact VARCHAR(50)
    )
""")
cursor.execute("""
    CREATE TABLE IF NOT EXISTS food_listings (
        Food_ID INT PRIMARY KEY,
        Food_Name VARCHAR(255),
        Quantity INT,
        Expiry_Date DATE,       
        Provider_ID INT,
        Provider_Type VARCHAR(100),
        Location VARCHAR(255),
        Food_Type VARCHAR(100),
        Meal_Type VARCHAR(100),          
        FOREIGN KEY (Provider_ID) REFERENCES providers(Provider_ID)
    )
""")
cursor.execute("""
    CREATE TABLE IF NOT EXISTS claims (
        Claim_ID INT PRIMARY KEY,
        Food_ID INT,
        Receiver_ID INT,
        Status VARCHAR(50),
        Timestamp DATETIME,       
        FOREIGN KEY (Food_ID) REFERENCES food_listings(Food_ID),
        FOREIGN KEY (Receiver_ID) REFERENCES receivers(Receiver_ID)
    )
""")
db.commit()

In [420]:
# data_food_listings['Expiry_Date']
# data_providers['Provider_ID'].values

In [421]:
# data_claims['Timestamp']

In [422]:
data_food_listings['Expiry_Date'] = pd.to_datetime(data_food_listings['Expiry_Date'], format='%m/%d/%Y')
data_claims['Timestamp'] = pd.to_datetime(
    data_claims['Timestamp'], 
    format='%m/%d/%Y %H:%M'
)


In [423]:
#Insert data into tables
try:
    for _, row in data_providers.iterrows():
        cursor.execute("""
        INSERT INTO providers (Provider_ID, Name, Type, Address, City, Contact)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, tuple(row))
    for _, row in data_receivers.iterrows():
        cursor.execute("""
        INSERT INTO receivers (Receiver_ID, Name, Type, City, Contact)
        VALUES (%s, %s, %s, %s, %s)
    """, tuple(row))
    for _, row in data_food_listings.iterrows():
        cursor.execute("""
        INSERT INTO food_listings (Food_ID, Food_Name, Quantity, Expiry_Date, Provider_ID, Provider_Type, Location, Food_Type, Meal_Type)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, tuple(row))
    for _, row in data_claims.iterrows():
        cursor.execute("""
        INSERT INTO claims (Claim_ID, Food_ID, Receiver_ID, Status, Timestamp)
        VALUES (%s, %s, %s, %s, %s)
    """, tuple(row))      
    cursor.execute("COMMIT")

except MySQLdb.Error as e:
    print(f"Error inserting data: {e}")

    

Error inserting data: (1062, "Duplicate entry '1' for key 'providers.PRIMARY'")


If Lock wait timeout exceeded; try restarting transaction

In [430]:
cursor.execute('show processlist')
processes = cursor.fetchall()
for process in processes:
    print(process)

(5, 'event_scheduler', 'localhost', None, 'Daemon', 12911, 'Waiting on empty queue', None)
(37, 'root', 'localhost:53980', None, 'Query', 0, 'init', 'show processlist')


In [431]:
sleeping_ids = [37]  # the ones from SHOW PROCESSLIST
for pid in sleeping_ids:
    cursor.execute(f"kill {pid}")

OperationalError: (1317, 'Query execution was interrupted')

Questions to be Answered (SQL Queries & Analysis)


Food Providers & Receivers


1-How many food providers and receivers are there in each city?


In [452]:
data_concanate=pd.concat([data_providers, data_receivers], axis=0, ignore_index=True)

In [458]:
query='select city,COUNT(*) as Provider_Count  from providers GROUP BY City order by Provider_Count desc;'
cursor.execute(query)
results = cursor.fetchall()
# print(results)
pd.DataFrame(results, columns=['City','Provider_Count'])


,City,Provider_Count
0,New Carol,3
1,South Christopherborough,3
2,New Amanda,2
3,North Kevinhaven,2
4,Lake Kyle,2
...,...,...
958,Manningshire,1
959,East Rossside,1
960,Joshuastad,1
961,Stevenchester,1


In [459]:
query='select city,COUNT(*) as receiver_Count  from receivers GROUP BY City order by receiver_Count desc;'
cursor.execute(query)
results = cursor.fetchall()
# print(results)
pd.DataFrame(results, columns=['City','receivers_Count'])

,City,receivers_Count
0,New Christopher,3
1,Spencermouth,2
2,Moorechester,2
3,Lake Mary,2
4,Smithshire,2
...,...,...
961,Lanechester,1
962,New Steven,1
963,South Sandra,1
964,Lake Jeffery,1


Which type of food provider (restaurant, grocery store, etc.) contributes the most food?

In [460]:
data_food_listings.head(2)

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,2025-03-17,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,2025-03-24,791,Grocery Store,West James,Non-Vegetarian,Dinner


In [462]:
query='select Provider_Type	,sum(Quantity)  from food_listings group by Provider_Type ;'
cursor.execute(query)
results = cursor.fetchall()
pd.DataFrame(results, columns=['Provider_Type','Total_Quantity'])

,Provider_Type,Total_Quantity
0,Grocery Store,6059
1,Catering Service,6116
2,Restaurant,6923
3,Supermarket,6696


What is the contact information of food providers in a specific city?


In [ ]:
data_providers.head(2)

,Provider_ID,Name,Type,Address,City,Contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480
1,2,"Nielsen, Johnson and Fuller",Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",East Sheena,+1-925-283-8901x6297


In [505]:
# data_providers['City'].value_counts()
query="select Name ,Contact, address from providers where city='South Christopherborough';"
cursor.execute(query)
results=cursor.fetchall()
pd.DataFrame(results)

,0,1,2
0,"Harris, Perez and Willis",581-281-6112,"549 Moody Road Suite 873\nJacobshire, VT 80966"
1,Torres-Dunlap,057.342.1300x473,Unit 1623 Box 5952\nDPO AA 55518
2,Brown and Sons,(352)481-4738,"725 Garner Locks\nAndrewchester, MI 49979"


Which receivers have claimed the most food

In [518]:
# data_claims['Receiver_ID'].value_counts()
data_claims.head(2)

,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,2025-03-05 05:26:00
1,2,353,391,Cancelled,2025-03-11 10:24:00


In [522]:
data_food_listings.head(2)
# data_food_listings['Food_Type'].value_counts()

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,2025-03-17,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,2025-03-24,791,Grocery Store,West James,Non-Vegetarian,Dinner


In [537]:
query = """
SELECT r.Receiver_ID, r.Name, COUNT(c.Claim_ID) AS Claim_Count
FROM claims c
JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
GROUP BY r.Receiver_ID, r.Name
ORDER BY Claim_Count DESC
LIMIT 5;
"""
cursor.execute(query)
results = cursor.fetchall()
import pandas as pd
df = pd.DataFrame(results, columns=["Receiver_ID", "Name", "Claim_Count"])
print(df)


   Receiver_ID               Name  Claim_Count
0          800     Anthony Garcia            5
1          276       Scott Hunter            5
2          371  William Frederick            5
3          742       Matthew Webb            5
4          301   Kristina Simpson            4


What is the total quantity of food available from all providers?


In [524]:
data_food_listings.head(2)

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,2025-03-17,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,2025-03-24,791,Grocery Store,West James,Non-Vegetarian,Dinner


In [ ]:
query='select sum(Quantity) from food_listings;'
cursor.execute(query)
results = cursor.fetchall()
pd.DataFrame(results,columns=['Total_Food_Quantity'])
# print("Total Food Quantity Available:",results[0][0])


Total Food Quantity Available: 25794


Which city has the highest number of food listings?


In [535]:
query = """
SELECT Location AS City, COUNT(*) AS FoodListing_Count
FROM food_listings
GROUP BY Location
ORDER BY FoodListing_Count DESC
LIMIT 1;
"""
cursor.execute(query)
# result = cursor.fetchone()
# print(f"City with the highest number of food listings: {result[0]}")
# print(f"Number of food listings: {result[1]}")
results = cursor.fetchall()
pd.DataFrame(results, columns=['City','FoodListing_Count'])

,City,FoodListing_Count
0,New Carol,6


What are the most commonly available food types?

In [539]:
query="select Food_Type, count(*) as Most_Avilable from  food_listings group by Food_Type  limit 1"
cursor.execute(query)
results = cursor.fetchall()
pd.DataFrame(results, columns=['Food_Type','Most_Avilable'])

,Food_Type,Most_Avilable
0,Non-Vegetarian,330


. How many food claims have been made for each food item?

In [543]:
data_food_listings.head(2)

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,2025-03-17,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,2025-03-24,791,Grocery Store,West James,Non-Vegetarian,Dinner


In [542]:
data_claims.head(2)


,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,2025-03-05 05:26:00
1,2,353,391,Cancelled,2025-03-11 10:24:00


In [546]:
query="""
select f.Food_Name, count(c.food_id)   from claims as c join food_listings as f
on c.Food_ID=f.Food_ID
group by f.Food_Name
order by count(c.food_id) desc

"""
cursor.execute(query)
results = cursor.fetchall()
pd.DataFrame(results, columns=['Food_Name','Claim_Count'])

,Food_Name,Claim_Count
0,Rice,122
1,Soup,114
2,Dairy,110
3,Fish,108
4,Salad,106
5,Chicken,102
6,Bread,94
7,Pasta,87
8,Vegetables,86
9,Fruits,71


Which provider has had the highest number of successful food claims?


In [548]:
data_food_listings.head(2)

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,2025-03-17,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,2025-03-24,791,Grocery Store,West James,Non-Vegetarian,Dinner


In [549]:
data_claims.head(2)


,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,2025-03-05 05:26:00
1,2,353,391,Cancelled,2025-03-11 10:24:00


In [ ]:
query='''
select f.Provider_ID ,count(c.Claim_ID) from claims c join food_listings f
on c.Food_ID=f.Food_ID
group by f.Provider_ID
limit 1
'''
cursor.execute(query)
results = cursor.fetchall()
pd.DataFrame(results, columns=['Provider_ID','Total_Claims'])

,Provider_ID,Total_Claims
0,1,3


. What percentage of food claims are completed vs. pending vs. canceled?

In [570]:
data_claims.head(2)


,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,2025-03-05 05:26:00
1,2,353,391,Cancelled,2025-03-11 10:24:00


In [581]:
query="select Status, count(*) from claims group by Status "
cursor.execute(query)
results = cursor.fetchall()
df=pd.DataFrame(results, columns=['Status','Count'])
df['percentage']= (df['Count']/df['Count'].sum())*100
df


,Status,Count,percentage
0,Pending,325,32.5
1,Cancelled,336,33.6
2,Completed,339,33.9


       11. What is the average quantity of food claimed per receiver?


In [586]:
data_food_listings.head(1)

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,2025-03-17,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast


In [585]:
data_claims.head(1)


,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,2025-03-05 05:26:00


In [587]:
query=" select c.Receiver_ID, avg(Quantity) from claims c join food_listings f on c.food_id=f.food_id group by  c.Receiver_ID "
cursor.execute(query)
results = cursor.fetchall()
pd.DataFrame(results, columns=['Receiver_ID','Avg_Quantity'])

,Receiver_ID,Avg_Quantity
0,908,25.6667
1,391,12.0000
2,492,37.0000
3,933,24.5000
4,229,14.0000
...,...,...
619,943,23.0000
620,984,26.0000
621,211,28.0000
622,746,2.0000


       12. Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?


In [593]:
query="select f.Meal_Type, count(*) from claims c join food_listings f on c.food_id=f.food_id group by  f.Meal_Type"
cursor.execute(query)
results = cursor.fetchall()
pd.DataFrame(results, columns=['Meal_Type','Count'])

,Meal_Type,Count
0,Breakfast,278
1,Snacks,240
2,Dinner,232
3,Lunch,250


What is the total quantity of food donated by each provider?

In [599]:
query="select f.Provider_ID, sum(f.Quantity) from claims c join food_listings f on c.food_id=f.food_id group by  f.Provider_ID	order by sum(f.Quantity) desc	"
cursor.execute(query)
results = cursor.fetchall()
pd.DataFrame(results, columns=['Provider_ID','Total_Quantity'])

,Provider_ID,Total_Quantity
0,752,348
1,315,334
2,384,299
3,709,228
4,850,217
...,...,...
467,762,2
468,186,2
469,876,2
470,184,1
